# Compile SEDflow posteriors and MCMC posteriors for NSA galaxies

In [1]:
import os
import h5py
import numpy as np
from tqdm.notebook import tqdm, trange

from sedflow import obs as Obs
from sedflow import train as Train

from provabgs import infer as Infer
from provabgs import models as Models

/home/chhahn/projects/provabgs/src/provabgs/models.py:23: UserWarning: import error with fsps; only use emulators
  warnings.warn('import error with fsps; only use emulators')


In [2]:
sample = 'toy'
itrain = 2
nhidden = 500
nblocks = 15

In [3]:
# u, g, r, i, z, sigma_u, sigma_g, sigma_r, sigma_i, sigma_z, redshift
nsa, igals = Obs.NSA()
y_nsa = Obs.load_nsa_data(test_set=False)
finite = np.all(np.isfinite(y_nsa[:,:5]), axis=1)

In [4]:
print('%i galaxies' % np.sum(finite))

33884 galaxies


In [5]:
prior = Train.prior_default()

In [6]:
# sps model
m_sps= Train.SPSmodel_default(emulator=True)

input parameters : logmstar, beta1_sfh, beta2_sfh, beta3_sfh, beta4_sfh, fburst, tburst, gamma1_zh, gamma2_zh, dust1, dust2, dust_index


In [7]:
# compile
posts, fails = [], []
logsfrs, logzmws = [], []
for ichunk in range(34):
    finite_chunk =  finite[ichunk*1000:(ichunk+1)*1000]

    fpost = os.path.join(Train.data_dir(), 'anpe_thetaunt_magsigz.%s.%ix%i.%i.nsa%iof34.samples.npy' % (sample, nhidden, nblocks, itrain, ichunk))
    print(fpost)
    _post = np.load(fpost)[finite_chunk,:,:]

    fail = (np.sum(np.sum(_post, axis=2), axis=1) == 0)

    igals = np.arange(y_nsa.shape[0])[ichunk*1000:(ichunk+1)*1000][finite_chunk][fail]
    iigals = np.arange(_post.shape[0])[fail]

    post = np.zeros((_post.shape[0], _post.shape[1], 12))
    post[~fail,:] = prior.transform(_post[~fail])

    if np.sum(fail) > 0:
        for iigal, igal in zip(iigals, igals):
            fgal = os.path.join(Train.data_dir(), 'nsa_fail', 'mcmc.nsa.%i.hdf5' % igal)
            if os.path.isfile(fgal):
                gal = h5py.File(fgal, 'r')
                chain_i = Train.flatten_chain(gal['mcmc_chain'][...][2000:,:,:])[-10000:,:]
                post[iigal,:,:] = chain_i
    posts.append(post)
    fails.append(fail)
    
    # derived galaxy properties
    z_chunk = y_nsa[:,-1][ichunk*1000:(ichunk+1)*1000][finite_chunk]

    logsfr = np.zeros((post.shape[0], post.shape[1]))
    logzmw = np.zeros((post.shape[0], post.shape[1]))
    for ii in trange(post.shape[0]):
        thetas_sps = post[ii,:]
        logsfr[ii,:] = np.log10(np.array(m_sps.avgSFR(thetas_sps, zred=float(z_chunk[ii]), dt=1.)))
        logzmw[ii,:] = np.log10(np.array(m_sps.Z_MW(thetas_sps, zred=float(z_chunk[ii])))).flatten()
    logsfrs.append(logsfr)
    logzmws.append(logzmw)

    #prop[ii,:,:] = Train.thetas2props(post[ii,:], np.repeat(z_chunk[ii], post[ii,:].shape[0]).astype(float))

posts = np.concatenate(posts)
fails = np.concatenate(fails)
logsfrs = np.concatenate(logsfrs)
logzmws = np.concatenate(logzmws)


/scratch/network/chhahn/sedflow/anpe_thetaunt_magsigz.toy.500x15.2.nsa0of34.samples.npy


  0%|          | 0/1000 [00:00<?, ?it/s]

/scratch/network/chhahn/sedflow/anpe_thetaunt_magsigz.toy.500x15.2.nsa1of34.samples.npy


  0%|          | 0/1000 [00:00<?, ?it/s]

/scratch/network/chhahn/sedflow/anpe_thetaunt_magsigz.toy.500x15.2.nsa2of34.samples.npy


...

  0%|          | 0/1000 [00:00<?, ?it/s]

/scratch/network/chhahn/sedflow/anpe_thetaunt_magsigz.toy.500x15.2.nsa32of34.samples.npy


  0%|          | 0/1000 [00:00<?, ?it/s]

/scratch/network/chhahn/sedflow/anpe_thetaunt_magsigz.toy.500x15.2.nsa33of34.samples.npy


  0%|          | 0/887 [00:00<?, ?it/s]

In [8]:
# write out
fsedflow = h5py.File(os.path.join(Train.data_dir(), 'nsa.sedflow.hdf5'), 'w')

fsedflow.create_dataset('NSAID', data=nsa['NSAID'][finite])
obs = ['mag_u', 'mag_g', 'mag_r', 'mag_i', 'mag_z', 'sigma_u', 'sigma_g', 'sigma_r', 'sigma_i', 'sigma_z', 'redshift']
for i, o in enumerate(obs):
    fsedflow.create_dataset(o, data=y_nsa[finite,i])

params = ['log_mstar', 'beta1', 'beta2', 'beta3', 'beta4', 'fburst', 'tburst', 'log_gamma1', 'log_gamma2', 'tau_bc', 'tau_ism', 'n_dust']
for i, param in enumerate(params):
    fsedflow.create_dataset(param, data=posts[:,:,i].astype(np.float32))

fsedflow.create_dataset('sedflow', data=~fails)    

# write out properties
fsedflow.create_dataset('log_sfr_1gyr', data=logsfrs.astype(np.float32))
fsedflow.create_dataset('log_z_mw', data=logzmws.astype(np.float32))

fsedflow.close()

In [11]:
fsedflow = h5py.File(os.path.join(Train.data_dir(), 'nsa.sedflow.hdf5'), 'r')

nsa_id = fsedflow['NSAID'][...]

log_sfr_1gyr = fsedflow['log_sfr_1gyr'][...]

In [14]:
fsedflow['tburst'][...].shape

(33884, 10000)

In [12]:
posts = []
props = []
fails = []
for ichunk in range(2):
    fpost = os.path.join(Train.data_dir(), 'anpe_thetaunt_magsigz.%s.%ix%i.%i.nsa%iof34.samples.npy' % (sample, nhidden, nblocks, itrain, ichunk))
    print(fpost)
    _post = np.load(fpost)
    fail = (np.sum(np.sum(_post, axis=2), axis=1) == 0)
    
    igals = np.arange(y_nsa.shape[0])[ichunk*1000:(ichunk+1)*1000][fail]
    iigals = np.arange(_post.shape[0])[fail]

    post = np.zeros((_post.shape[0], _post.shape[1], 12))
    post[~fail,:] = prior.transform(_post[~fail])
    
    if np.sum(fail) > 0: 
        for iigal, igal in zip(iigals, igals): 
            fgal = os.path.join(Train.data_dir(), 'nsa_fail', 'mcmc.nsa.%i.hdf5' % igal)
            if os.path.isfile(fgal): 
                gal = h5py.File(fgal, 'r')
                chain_i = Train.flatten_chain(gal['mcmc_chain'][...][2000:,:,:])[-10000:,:]
                post[iigal,:,:] = chain_i
    fails.append(fail)
    posts.append(post)
    
    z_chunk = y_nsa[:,-1][np.arange(y_nsa.shape[0])[ichunk*1000:(ichunk+1)*1000]]
    
    prop = np.zeros((post.shape[0], post.shape[1], 4))
    for ii in range(post.shape[0]): 
        prop[ii,:,:] = Train.thetas2props(post[ii,:], np.repeat(z_chunk[ii], post[ii,:].shape[0]).astype(float))
    
posts = np.concatenate(posts)
fails = np.concatenate(fails)

/scratch/network/chhahn/sedflow/anpe_thetaunt_magsigz.toy.500x15.2.nsa0of34.samples.npy
[0.02122228 0.02122228 0.02122228 ... 0.02122228 0.02122228 0.02122228]
input parameters : logmstar, beta1_sfh, beta2_sfh, beta3_sfh, beta4_sfh, fburst, tburst, gamma1_zh, gamma2_zh, dust1, dust2, dust_index
[0.04808582 0.04808582 0.04808582 ... 0.04808582 0.04808582 0.04808582]
input parameters : logmstar, beta1_sfh, beta2_sfh, beta3_sfh, beta4_sfh, fburst, tburst, gamma1_zh, gamma2_zh, dust1, dust2, dust_index
[0.02196459 0.02196459 0.02196459 ... 0.02196459 0.02196459 0.02196459]
input parameters : logmstar, beta1_sfh, beta2_sfh, beta3_sfh, beta4_sfh, fburst, tburst, gamma1_zh, gamma2_zh, dust1, dust2, dust_index
[0.03580831 0.03580831 0.03580831 ... 0.03580831 0.03580831 0.03580831]
input parameters : logmstar, beta1_sfh, beta2_sfh, beta3_sfh, beta4_sfh, fburst, tburst, gamma1_zh, gamma2_zh, dust1, dust2, dust_index
[0.05248393 0.05248393 0.05248393 ... 0.05248393 0.05248393 0.05248393]
input pa

[0.04713756 0.04713756 0.04713756 ... 0.04713756 0.04713756 0.04713756]
input parameters : logmstar, beta1_sfh, beta2_sfh, beta3_sfh, beta4_sfh, fburst, tburst, gamma1_zh, gamma2_zh, dust1, dust2, dust_index
[0.04526348 0.04526348 0.04526348 ... 0.04526348 0.04526348 0.04526348]
input parameters : logmstar, beta1_sfh, beta2_sfh, beta3_sfh, beta4_sfh, fburst, tburst, gamma1_zh, gamma2_zh, dust1, dust2, dust_index
[0.03270648 0.03270648 0.03270648 ... 0.03270648 0.03270648 0.03270648]
input parameters : logmstar, beta1_sfh, beta2_sfh, beta3_sfh, beta4_sfh, fburst, tburst, gamma1_zh, gamma2_zh, dust1, dust2, dust_index
[0.04281429 0.04281429 0.04281429 ... 0.04281429 0.04281429 0.04281429]
input parameters : logmstar, beta1_sfh, beta2_sfh, beta3_sfh, beta4_sfh, fburst, tburst, gamma1_zh, gamma2_zh, dust1, dust2, dust_index
[0.03274609 0.03274609 0.03274609 ... 0.03274609 0.03274609 0.03274609]
input parameters : logmstar, beta1_sfh, beta2_sfh, beta3_sfh, beta4_sfh, fburst, tburst, gamma1_

KeyboardInterrupt: 

In [ ]:
fsedflow = h5py.File(os.path.join(Train.data_dir(), 'nsa.sedflow.hdf5'), 'w')

params = ['logMstar', 'beta1', 'beta2', 'beta3', 'beta4', 'fburst', 'tburst', 'log_gamma1', 'log_gamma2', 'dust1', 'dust2', 'dust_index']

for i, param in enumerate(params): 
    fsedflow.create_dataset(params, data=posts[:,:,i])
fsedflow.close()